# Modelos de Predicción del Potencial de Licuación - FONDEF ID16I20157

## *Ejemplos de Aplicación*

En este Notebook se ejemplifica la aplicación de los modelos de predicción del potencial de licuación desarrollados para el proyecto FONDEF ID16I20157.

Se entrega un ejemplo de aplicación para el modelo de CPT, y un ejemplo para el modelo de CPT.

En cuanto a las medidas de intensidad utilizadas, para ambos casos se utilizan los modelos de Montalva et al. (2017)<sup>1</sup> y Montalva et al. (2022)<sup>2</sup>.

Dichos modelos, junto con los modelos de predicción del potencial de licuación, se encuentran implementados en el GeoPortal de Peligrosidad Sísmica de Chile, el que está disponible en <a href="http://geop.servequake.com/cyclo/geoportal/app/" target="_blank">este link</a>.

### Referencias

1. Montalva, A., Bastías, N., y Rodriguez‐Marek, A. (2017) Ground‐Motion Prediction Equation for the Chilean Subduction Zone. *Bulletin of the Seismological Society of America*, *107*(2), 901–911. doi: https://doi.org/10.1785/0120160221

2. Montalva, A., Bastías, N., Montalva, A., Bastías, N.  y Leyton, F. (2021) Strong Ground Motion Prediction Model for PGV and Spectral Velocity for the Chilean Subduction Zone. *Bulletin of the Seismological Society of America*, *112*(1), 348–360. doi: https://doi.org/10.1785/0120210037

3. Lasley, S., Green, R., y Rodriguez-Marek, A. (2016). New Stress Reduction Coefficient Relationship for Liquefaction Triggering Analyses. *Journal of Geotechnical and Geoenvironmental Engineering*, *142*(11). doi: https://doi.org/10/f8825b

4. Lasley, S., Green, R., y Rodriguez-Marek, A. (2017). Number of Equivalent Stress Cycles for Liquefaction Evaluations in Active Tectonic and Stable Continental Regimes. *Journal of Geotechnical and Geoenvironmental Engineering*, *143*(4). doi: https://doi.org/10/f937hd

5. Boulanger, R., y Idriss, I. (2014). *CPT and SPT Based Liquefaction Triggering Procedures*. Report UCD/CGM-14/01. University of California: Department of Civil and Environmental Engineering, Center for Geotechnical Modeling.


# Aplicación del Modelo basado en CPT

En este bloque se ejemplifica la aplicación del modelo de predicción de licuación basado en CPT.


## Inputs

Para utilizar el modelo, se requieren los siguientes inputs:

### Parámetros geotécnicos

* Profundidad del estrato licuable (*z*) en *m*.
    
* Tensión vertical total (*$\sigma$<sub>v0</sub>*) en *kPa*.
     
* Tensión vertical efectiva (*$\sigma$'<sub>v0</sub>*) en *kPa*.

* Resistencia por punta obtenida por CPT normalizada y corregida por finos (*q<sub>c1Ncs</sub>*) 

### Parámetros geofísicos

* Frecuencia predominante del sitio (*f<sub>0</sub>*) en *Hz*.

* Velocidad de onda de corte promedio en los primeros 12 metros (*V<sub>s12</sub>*) en *m/s*.

* Velocidad de onda de corte promedio en los primeros 30 metros (*V<sub>s30</sub>*) en *m/s*.

### Parámetros sísmicos

* Magnitud de momento (*M<sub>w</sub>*)

* Tipo de evento sísmico (*Subd*)

* Aceleración máxima en superficie (*PGA*) en *g*.

* Velocidad máxima en superficie (*PGA*) en *cm/s*.


In [20]:
# En primera instancia, se importan algunos modulos necesarios para correr el código

import numpy as np

from scipy.stats import norm

# A continuación se definen los inputs para aplicar el modelo

# Parámetros geotécnicos

z = 8.97 #Profundidad en [m]

st_tot = 134.1 #Tensión vertical total en [kPa]

st_eff = 109.91 #Tensión vertical efectiva en [kPa]

q_c1Ncs = 131.01 #Resistencia por punta obtenida por CPT normalizada y corregida


# Parámetros geofísicos

Vs12 = 206.8 # Velocidad de onda de corte promedio en los primeros 12 metros en [m/s]

Vs30 = 234.8 # Velocidad de onda de corte promedio en los primeros 12 metros en [m/s]

f0 = 1.25 # Frecuencia predominante en [Hz]


# Parámetros geofísicos

Subd = 1 # Tipo de evento: (1) Interface (0) Otro

Mw = 8.8 # Velocidad de onda de corte promedio en los primeros 12 metros en [m/s]

PGA = 0.292 # Aceleración máxima en superficie en [g]

PGV = 64.133 #  Velocidad máxima en superficie en [cm/s]

# Tanto PGA como PGV se estimaron en el GeoPortal, utilizando los siguientes parámetros complementarios
#
#    Distancia = 75 km
#
#    Profundidad focal = 25 km
#
#    Antearco / Trasarco = 0
#
#    Peak Claro = 1
#
#    epsilon = 0



## Cálculos Intermedios

A continuación se ilustra paso a paso la aplicación del modelo:

La ecuación que describe la demanda cíclica (CSR) es:

$$CSR_{Mod} = S_{Mod} \cdot CSR$$

Donde:

$$CSR = 0.65\cdot\frac{PGA}{g}\cdot\frac{\sigma_{v0}}{\sigma'_{v0}}\cdot \frac{r_{d}}{MSF\cdot K_{\sigma}}$$



El parámetro $K_{\sigma}$ se calcula con el Modelo de Boulanger e Idriss (2014):

In [21]:
p_atm = 101.325 # Presión atmosférica

q = np.where(q_c1Ncs > 211, 211, q_c1Ncs) # Verificación de que q_c1Ncs está dentro de los límites de aplicación de la fórmula

Csig = 1/ (37.3 - 8.27 * (q ** 0.264)) # Factor intermedio

Csig = np.where(Csig > 0.3, 0.3, Csig) # Verificación de que Csig se encuentra dentro de los límites permitidos

Ksig = 1 - Csig * np.log(st_eff / p_atm) # Factor de corrección por confinamiento

Ksig = np.where(Ksig > 1.1, 1.1, Ksig) # Verificación de que Ksig se encuentra dentro de los límites permitidos

print(Ksig)


0.9889268673972329


El parámetro $r_{d}$ se calcula con el Modelo de Lasley et al (2016):

In [22]:
alpha = np.exp(-3.793 + 0.4016 * Mw - 0.001405 * Vs12) # Parámetro intermedio
    
beta = np.exp(-1.380 + 0.3276 * Mw + 0.01332 * Vs12) # Parámetro intermedio 
    
rd = (1 - alpha) * np.exp(-z / beta) + alpha # Factor de reducción de esfuerzos
    
print(rd)


0.9495806623027131


El parámetro $MSF$ se calcula con el Modelo de Lasley et al (2017):

In [23]:
neq = np.exp(0.4605 - 0.4082 * np.log(PGA) + 0.2332 * Mw) # N° de ciclos equivalentes
    
MSF = (14 / neq) ** 0.34 # Magnitude Scaling Factor

MSF = np.where(MSF > 2.02, 2.02, MSF) # Verificación de que MSF se encuentra dentro de los límites permitidos

print(MSF)


0.8799650036493369


Una vez obtenidos los parámetros $K_{\sigma}$, $r_{d}$ y $MSF$ se puede calcular la demanda cíclica sin ajustar ($CSR$):

In [24]:
CSR = 0.65 * PGA * (st_tot / st_eff) * rd / (MSF * Ksig) # Demanda cíclica sin ajustar

print(CSR)

0.2526911682976022


La demanda cíclica obtenida anteriormente se ajustará para eventos de subducción de interface:

In [25]:
if Subd == 1: # Si el terremoto es de interface, se ajusta la demanda cíclica

    Smod = (((1 / f0 )** 2.6165) *
            ((10 / PGV) ** 2.2694) * 
            ((Vs30 / Vs12) ** 1.1057) * 
            np.exp(1.5645 * (np.log(PGV / 10)) ** 2)
            ) #Factor de modificación para eventos de interface

else: # Si el terremoto no es de interface, no se ajusta la demanda cíclica

    Smod = 1

CSR_mod = CSR * Smod

print(CSR_mod)


0.5308120015319138


La ecuación que describe la resistencia cíclica (CRR) es:

$$CRR = \exp\left(1.1002\cdot\left[\frac{q_{c1Ncs}}{113}+\left(\frac{q_{c1Ncs}}{1000}\right)^2-\left(\frac{q_{c1Ncs}}{140}\right)^3+\left(\frac{q_{c1Ncs}}{137}\right)^4\right] - 2.8433\right)$$

La resistencia cíclica se obtiene a continuación:

In [26]:
CRR = np.exp(1.1002 * ((q / 113) + (q / 1000) ** 2 - (q / 140) ** 3 + (q / 137) ** 4) - 2.8433)

print(CRR)

0.21644947638113113


Una vez calculadas tanto la demanda cíclica como la resistencia cíclica, puede obtnerse tanto la probabilidad de licuar ($P_{Liq}$) como el factor de seguridad ($FS_{Liq}$).

La probabilidad de licuar se obtiene con la siguiente ecuación:

$$P_{Liq} = \Phi\left[ - \frac{\ln(CRR) - \ln(CSR_{Mod})}{0.5502}\right] $$

Donde $\Phi$ es la función de distribución de probabilidad de la distribución normal.

El factor de seguridad a licuar se obtiene con la siguiente ecuación:

$$FS_{Liq} = \frac{CRR\cdot\exp\left[0.5502\cdot\Phi^{-1}\left(0.25\right)\right]}{CSR_{Mod}} $$

La probabilidad de licuar se obtiene a continuación:

In [27]:
Pliq = norm.cdf(- (np.log(CRR) - np.log(CSR_mod)) / 0.5502)

print(round(Pliq, 3))

0.948


 El factor de seguridad a licuar se obtiene a continuación:

In [28]:
CRR25 = CRR * np.exp(0.5502 * norm.ppf(0.25))
    
FSliq = CRR25 / CSR_mod

print(round(FSliq, 3))

0.281


# Aplicación del Modelo basado en SPT

En este bloque se ejemplifica la aplicación del modelo de predicción de licuación basado en SPT.


## Inputs

Para utilizar el modelo, se requieren los siguientes inputs:

### Parámetros geotécnicos

* Profundidad del estrato licuable (*z*) en *m*.
    
* Tensión vertical total (*$\sigma$<sub>v0</sub>*) en *kPa*.
     
* Tensión vertical efectiva (*$\sigma$'<sub>v0</sub>*) en *kPa*.

* N° de golpes obtenidos con SPT normalizados y corregidos por finos (*(N<sub>1</sub>)<sub>60cs</sub>*) 

### Parámetros geofísicos

* Frecuencia predominante del sitio (*f<sub>0</sub>*) en *Hz*.

* Velocidad de onda de corte promedio en los primeros 12 metros (*V<sub>s12</sub>*) en *m/s*.

* Velocidad de onda de corte promedio en los primeros 30 metros (*V<sub>s30</sub>*) en *m/s*.

### Parámetros sísmicos

* Magnitud de momento (*M<sub>w</sub>*)

* Tipo de evento sísmico (*Subd*)

* Aceleración máxima en superficie (*PGA*) en *g*.

* Velocidad máxima en superficie (*PGA*) en *cm/s*.


In [29]:
# En primera instancia, se importan algunos modulos necesarios para correr el código

import numpy as np

from scipy.stats import norm

# A continuación se definen los inputs para aplicar el modelo

# Parámetros geotécnicos

z = 8.97 #Profundidad en [m]

st_tot = 134.1 #Tensión vertical total en [kPa]

st_eff = 109.91 #Tensión vertical efectiva en [kPa]

N1_60cs = 12.5 # N° de golpes corregidos y normalizados


# Parámetros geofísicos

Vs12 = 206.8 # Velocidad de onda de corte promedio en los primeros 12 metros en [m/s]

Vs30 = 234.8 # Velocidad de onda de corte promedio en los primeros 12 metros en [m/s]

f0 = 1.25 # Frecuencia predominante en [Hz]


# Parámetros geofísicos

Subd = 1 # Tipo de evento: (1) Interface (0) Otro

Mw = 8.8 # Velocidad de onda de corte promedio en los primeros 12 metros en [m/s]

PGA = 0.292 # Aceleración máxima en superficie en [g]

PGV = 64.133 #  Velocidad máxima en superficie en [cm/s]

# Tanto PGA como PGV se estimaron en el GeoPortal, utilizando los siguientes parámetros complementarios
#
#    Distancia = 75 km
#
#    Profundidad focal = 25 km
#
#    Antearco / Trasarco = 0
#
#    Peak Claro = 1
#
#    epsilon = 0



## Cálculos Intermedios

A continuación se ilustra paso a paso la aplicación del modelo:

La ecuación que describe la demanda cíclica (CSR) es:

$$CSR_{Mod} = S_{Mod} \cdot CSR$$

Donde:

$$CSR = 0.65\cdot\frac{PGA}{g}\cdot\frac{\sigma_{v0}}{\sigma'_{v0}}\cdot \frac{r_{d}}{MSF\cdot K_{\sigma}}$$



El parámetro $K_{\sigma}$ se calcula con el Modelo de Boulanger e Idriss (2014):

In [30]:
p_atm = 101.325 # Presión atmosférica

N = np.where(N1_60cs > 37, 37, N1_60cs) # Verificación de que q_c1Ncs está dentro de los límites de aplicación de la fórmula

Csig = 1/ (18.9 - 2.55 * np.sqrt(N)) # Factor intermedio

Csig = np.where(Csig > 0.3, 0.3, Csig) # Verificación de que Csig se encuentra dentro de los límites permitidos

Ksig = 1 - Csig * np.log(st_eff / p_atm) # Factor de corrección por confinamiento

Ksig = np.where(Ksig > 1.1, 1.1, Ksig) # Verificación de que Ksig se encuentra dentro de los límites permitidos

print(Ksig)


0.9917720073185737


El parámetro $r_{d}$ se calcula con el Modelo de Lasley et al (2016):

In [31]:
alpha = np.exp(-3.793 + 0.4016 * Mw - 0.001405 * Vs12) # Parámetro intermedio
    
beta = np.exp(-1.380 + 0.3276 * Mw + 0.01332 * Vs12) # Parámetro intermedio 
    
rd = (1 - alpha) * np.exp(-z / beta) + alpha # Factor de reducción de esfuerzos
    
print(rd)


0.9495806623027131


El parámetro $MSF$ se calcula con el Modelo de Lasley et al (2017):

In [32]:
neq = np.exp(0.4605 - 0.4082 * np.log(PGA) + 0.2332 * Mw) # N° de ciclos equivalentes
    
MSF = (14 / neq) ** 0.34 # Magnitude Scaling Factor

MSF = np.where(MSF > 2.02, 2.02, MSF) # Verificación de que MSF se encuentra dentro de los límites permitidos

print(MSF)


0.8799650036493369


Una vez obtenidos los parámetros $K_{\sigma}$, $r_{d}$ y $MSF$ se puede calcular la demanda cíclica sin ajustar ($CSR$):

In [33]:
CSR = 0.65 * PGA * (st_tot / st_eff) * rd / (MSF * Ksig) # Demanda cíclica sin ajustar

print(CSR)

0.2519662620435554


La demanda cíclica obtenida anteriormente se ajustará para eventos de subducción de interface:

In [34]:
if Subd == 1: # Si el terremoto es de interface, se ajusta la demanda cíclica

    Smod = (((10 / PGV) ** 0.36332) * 
                ((Vs30 / Vs12) ** 2.26291) * 
                np.exp(-5.39071 * (np.log(Vs30 / Vs12)) ** 2)
                ) #Factor de modificación para eventos de interface

else: # Si el terremoto no es de interface, no se ajusta la demanda cíclica

    Smod = 1

CSR_mod = CSR * Smod

print(CSR_mod)


0.15673201603620715


La ecuación que describe la resistencia cíclica (CRR) es:

$$CRR = \exp\left(1.5314\cdot\left[\frac{\left(N_{1}\right)_{60cs}}{113}+\left(\frac{\left(N_{1}\right)_{60cs}}{1000}\right)^2-\left(\frac{\left(N_{1}\right)_{60cs}}{140}\right)^3+\left(\frac{\left(N_{1}\right)_{60cs}}{137}\right)^4\right] - 3.2171\right)$$

La resistencia cíclica se obtiene a continuación:

In [35]:
CRR = np.exp(1.53139 * ((N1_60cs / 14.1) + (N1_60cs / 126) ** 2 - (N1_60cs / 23.6) ** 3 + (N1_60cs / 25.4) ** 4) - 3.21714)
    
print(CRR)

0.13776846061678857


Una vez calculadas tanto la demanda cíclica como la resistencia cíclica, puede obtnerse tanto la probabilidad de licuar ($P_{Liq}$) como el factor de seguridad ($FS_{Liq}$).

La probabilidad de licuar se obtiene con la siguiente ecuación:

$$P_{Liq} = \Phi\left[ - \frac{\ln(CRR) - \ln(CSR_{Mod})}{0.5755}\right] $$

Donde $\Phi$ es la función de distribución de probabilidad de la distribución normal.

El factor de seguridad a licuar se obtiene con la siguiente ecuación:

$$FS_{Liq} = \frac{CRR\cdot\exp\left[0.5755\cdot\Phi^{-1}\left(0.15\right)\right]}{CSR_{Mod}} $$

La probabilidad de licuar se obtiene a continuación:

In [36]:
Pliq = norm.cdf(- (np.log(CRR) - np.log(CSR_mod)) / 0.57547)

print(round(Pliq, 3))

0.589


 El factor de seguridad a licuar se obtiene a continuación:

In [37]:
CRR15 = CRR * np.exp(0.57547 * norm.ppf(0.15))
    
FSliq = CRR15 / CSR_mod

print(round(FSliq, 3))

0.484
